## Connecting to Presto

Presto ships with a CLI which supports autocompletion, history, progress bars and other useful features. For quickly testing queries the CLI is very helpful.

In a environment like a Jupyter notebook, we can use a Presto Python client. The Presto client library implements the Python DBAPI2.0 interface that is used by common database client libraries for querying MySQL, PostgreSQL and SQLite.

DBAPI2.0 defines a API with a `Connection`. Queries then happen with a `cursor`. Presto supports transaction. The level of isolation depends on the connectors involved in a query.

The three mandatory arguments to create a connection are *host*, *port*, and *user*.
Other arguments such as *source* allow to identify the origin of the query. A common use case is to use it to tell which service, tool, or code sent the query.

Let's create a connection:

In [3]:
import prestodb.dbapi as presto

conn = presto.Connection(host="localhost", port=8080, user="johnsnow")
cur = conn.cursor()
cur

In [ ]:
## Configuration

Presto's general configuration is documented in the [deployment](https://prestodb.github.io/docs/current/installation/deployment.html) page. There are 4 types of configuration files:
- Node Properties: to configure the coordinator (main server) and worker nodes.
- JVM Config: command line options for the Java Virtual Machine that runs Presto.
- Config Properties: configuration for the Presto server
- Catalog Properties: configuration for Connectors (data sources)

In the test, the configuration is in `coordinator/presto/etc/`. The main file to configure Presto is `config.properties`:

```
node.id=presto-master
node.environment=test

coordinator=true
node-scheduler.include-coordinator=true
http-server.http.port=8080
query.max-memory=256MB
query.max-memory-per-node=256MB
discovery-server.enabled=true
discovery.uri=http://localhost:8080
```

The discovery is what allows worker nodes to find the coordinator and register themselves. Then they will participate in the execution of queries.

## Catalogs

A catalog is mapped to a connector. The name of configuration file for a catalgo defines the catalog's name. Here `etc/catalog/timescaledb.properties` configures the `timescaledb` catalog. We could name it `events` or `users`:

```
connector.name=postgresql
connection-url=jdbc:postgresql://localhost:5433/db1
connection-user=johnsnow
connection-password=password
```

Adding a catalog is a simple as adding a file with the catalog properties and named after the catalog's name.


Below we list the available catalogs on the Presto cluster we are running:

In [4]:
cur.execute("SHOW catalogs")
cur.fetchall()

[['blackhole'],
 ['jmx'],
 ['memory'],
 ['system'],
 ['timescaledb'],
 ['tpcds'],
 ['tpch']]

## How Does Presto Execute a Query?

If you are curious about what Presto translate a SQL query to and what it will run, you can you `EXPLAIN`:

In [21]:
conn = presto.Connection(host="localhost", port=8080, user="johnsnow", catalog="timescaledb", schema="default")
cur.execute("explain (type DISTRIBUTED) SELECT * FROM timescaledb.public.table1")
plan = cur.fetchall()

print(plan[0][0])

Fragment 0 [SINGLE]
    Output layout: [id, field1]
    Output partitioning: SINGLE []
    Stage Execution Strategy: UNGROUPED_EXECUTION
    - Output[id, field1] => [id:varchar, field1:varchar]
            Estimates: {rows: ? (?), cpu: ?, memory: 0.00, network: ?}
        - RemoteSource[1] => [id:varchar, field1:varchar]

Fragment 1 [SOURCE]
    Output layout: [id, field1]
    Output partitioning: SINGLE []
    Stage Execution Strategy: UNGROUPED_EXECUTION
    - TableScan[TableHandle {connectorId='timescaledb', connectorHandle='timescaledb:public.table1:null:public:table1', layout='Optional[timescaledb:public.table1:null:public:table1]'}, grouped = false] => [id:varchar, field1:varchar]
            Estimates: {rows: ? (?), cpu: ?, memory: 0.00, network: 0.00}
            id := JdbcColumnHandle{connectorId=timescaledb, columnName=id, jdbcTypeHandle=JdbcTypeHandle{jdbcType=12, columnSize=2147483647, decimalDigits=0}, columnType=varchar, nullable=false}
            field1 := JdbcColumnHan

```
Fragment 0 [SINGLE]
    Output layout: [id, field1]
    Output partitioning: SINGLE []
    Stage Execution Strategy: UNGROUPED_EXECUTION
    - Output[id, field1] => [id:varchar, field1:varchar]
            Estimates: {rows: ? (?), cpu: ?, memory: 0.00, network: ?}
        - RemoteSource[1] => [id:varchar, field1:varchar]

Fragment 1 [SOURCE]
    Output layout: [id, field1]
    Output partitioning: SINGLE []
    Stage Execution Strategy: UNGROUPED_EXECUTION
    - TableScan[TableHandle {connectorId='timescaledb', connectorHandle='timescaledb:public.table1:null:public:table1', layout='Optional[timescaledb:public.table1:null:public:table1]'}, grouped = false] => [id:varchar, field1:varchar]
            Estimates: {rows: ? (?), cpu: ?, memory: 0.00, network: 0.00}
            id := JdbcColumnHandle{connectorId=timescaledb, columnName=id, jdbcTypeHandle=JdbcTypeHandle{jdbcType=12, columnSize=2147483647, decimalDigits=0}, columnType=varchar, nullable=false}
            field1 := JdbcColumnHandle{connectorId=timescaledb, columnName=field1, jdbcTypeHandle=JdbcTypeHandle{jdbcType=12, columnSize=2147483647, decimalDigits=0}, columnType=varchar, nullable=true}
```

In [20]:
cur.execute("explain SELECT * FROM timescaledb.public.table1")
plan = cur.fetchall()

print(plan[0][0])

- Output[id, field1] => [id:varchar, field1:varchar]
        Estimates: {rows: ? (?), cpu: ?, memory: 0.00, network: ?}
    - RemoteStreamingExchange[GATHER] => [id:varchar, field1:varchar]
            Estimates: {rows: ? (?), cpu: ?, memory: 0.00, network: ?}
        - TableScan[TableHandle {connectorId='timescaledb', connectorHandle='timescaledb:public.table1:null:public:table1', layout='Optional[timescaledb:public.table1:null:public:table1]'}] => [id:varchar, field1:varchar]
                Estimates: {rows: ? (?), cpu: ?, memory: 0.00, network: 0.00}
                id := JdbcColumnHandle{connectorId=timescaledb, columnName=id, jdbcTypeHandle=JdbcTypeHandle{jdbcType=12, columnSize=2147483647, decimalDigits=0}, columnType=varchar, nullable=false}
                field1 := JdbcColumnHandle{connectorId=timescaledb, columnName=field1, jdbcTypeHandle=JdbcTypeHandle{jdbcType=12, columnSize=2147483647, decimalDigits=0}, columnType=varchar, nullable=true}



```
- Output[id, field1] => [id:varchar, field1:varchar]
        Estimates: {rows: ? (?), cpu: ?, memory: 0.00, network: ?}
    - RemoteStreamingExchange[GATHER] => [id:varchar, field1:varchar]
            Estimates: {rows: ? (?), cpu: ?, memory: 0.00, network: ?}
        - TableScan[TableHandle {connectorId='timescaledb', connectorHandle='timescaledb:public.table1:null:public:table1', layout='Optional[timescaledb:public.table1:null:public:table1]'}] => [id:varchar, field1:varchar]
                Estimates: {rows: ? (?), cpu: ?, memory: 0.00, network: 0.00}
                id := JdbcColumnHandle{connectorId=timescaledb, columnName=id, jdbcTypeHandle=JdbcTypeHandle{jdbcType=12, columnSize=2147483647, decimalDigits=0}, columnType=varchar, nullable=false}
                field1 := JdbcColumnHandle{connectorId=timescaledb, columnName=field1, jdbcTypeHandle=JdbcTypeHandle{jdbcType=12, columnSize=2147483647, decimalDigits=0}, columnType=varchar, nullable=true}
```

## Create a Table in TimeScaleDB

Let's use the TimeScaleDB client to create the table. Then we will switch to Presto to manipulate the data:

In [19]:
cur.execute("CREATE TABLE IF NOT EXISTS timescaledb.public.ts_events (event varchar(1000000))")
cur.fetchall()
cur.execute("DESC timescaledb.public.ts_events")
for row in cur.fetchall():
    print("{table}: [{props}]".format(
        table=row[0],
        props=', '.join(str(i) for i in row[1:])))

event: [varchar(1000000), , ]


## Load Data in TimeScaleDB

Let's now load data from [GH Archive](http://www.gharchive.org/) into TimeScaleDB.
Each file from GH Archive contains lines of JSON structs that represent events from the public GitHub timeline, for example repository creation or code push.

Now that the table is create in TimeScaleDB, we can insert rows with Presto by using the existing `conn` object created above. You can open http://localhost:8080 to see the execution Presto queries.

In [11]:
import gzip
import io
import json
import re
import requests

# Load events happening between 4-5pm.
# Feel free to load more hours or more days.
# We limit the dataset to one hour here to not overload
# the machine that will run the queries as this tutorial
# is expected to run on a laptop.
# It is going to take some time. For the demo, i pre-loaded
# the data with the mysql client to avoid the overhead of creating
# Python objects.
zdata = requests.get("https://data.gharchive.org/2015-04-28-16.json.gz")
data = gzip.decompress(zdata.content)
rows = []

# load ``ROW_COUNT`` rows. Feel free to set a greater value if it
# works well in your environment. Using a small value on purpose
# to avoid loading data for a long time.
ROW_COUNT = 1000
cur = conn.cursor()
for n, line in enumerate(io.BytesIO(data)):
    row = line.strip().decode('utf8')
    sql = "INSERT INTO timescaledb.public.ts_events (event) VALUES ('{}')".format(row.replace("'", "''"))
    cur.execute(sql)
    cur.fetchall()
    if n == ROW_COUNT - 1:
        break

In [18]:
cur = conn.cursor()
cur.execute("SELECT event FROM timescaledb.public.ts_events LIMIT 1")
rows = cur.fetchall()
import json
parsed = json.loads(rows[0][0])
print(json.dumps(parsed, indent=4, sort_keys=True))

{
    "actor": {
        "avatar_url": "https://avatars.githubusercontent.com/u/126762?",
        "gravatar_id": "",
        "id": 126762,
        "login": "seifsallam",
        "url": "https://api.github.com/users/seifsallam"
    },
    "created_at": "2015-04-28T16:00:00Z",
    "id": "2761189792",
    "payload": {
        "before": "b0654ac2ccc6a0860587d118ca74e9af10fcac52",
        "commits": [
            {
                "author": {
                    "email": "73675debcd8a436be48ec22211dcf44fe0df0a64@ibotta.com",
                    "name": "Ben Limmer"
                },
                "distinct": true,
                "message": "Add Ember.run snippet\n\nBecause the run loop doesn't run automatically in test, a lot of times I need to insert a fun loop. This addition would be really handy!",
                "sha": "bae7eabf1ba43afbdef4e6023de0a7939986d7f6",
                "url": "https://api.github.com/repos/seifsallam/atom-ember-snippets/commits/bae7eabf1ba43afbdef4e6023de0a